<a href="https://colab.research.google.com/github/garciasergio94y/RNN/blob/LSTM_onehot_QC/LSTM_onehot_QC_R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
# Instalación y carga de librerías:

packages <- c("readr", "readxl", "purrr", "dplyr", "filesstrings",
              "stringr", "tidyr", "lubridate", "ggplot2", #"caret",
              "tictoc", "wavelets", "reticulate", "abind",
              "tensorflow", "tfdatasets", "keras")

# Función para instalar paquetes si no están ya instalados
install_packages <- function(package) {
  if (!require(package, character.only = TRUE)) {
    install.packages(package)
    }
}

# Aplicar la función para cada uno de los paquetes
lapply(packages, install_packages)

require(readr)
require(readxl)
require(purrr)
require(dplyr)
require(filesstrings)
require(stringr)
require(tidyr)
require(lubridate)
require(ggplot2)
require(caret)
require(wavelets)
require(tictoc)
require(reticulate)
require(abind)
require(tensorflow)
require(tfdatasets)
require(keras)


Loading required package: readxl

Loading required package: purrr

Loading required package: filesstrings

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘filesstrings’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘checkmate’, ‘strex’


Loading required package: stringr

Loading required package: tidyr

Loading required package: ggplot2

Loading required package: tictoc

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘tictoc’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Loading required package: wavelets

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘wavelets’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ i

[[1]]
NULL

[[2]]
NULL

[[3]]
NULL

[[4]]
NULL

[[5]]
NULL

[[6]]
NULL

[[7]]
NULL

[[8]]
NULL

[[9]]
NULL

[[10]]
NULL

[[11]]
NULL

[[12]]
NULL

[[13]]
NULL

[[14]]
NULL

[[15]]
NULL

[[16]]
NULL

Loading required package: filesstrings


Attaching package: ‘filesstrings’


The following object is masked from ‘package:dplyr’:

    all_equal


Loading required package: caret

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘caret’”
Loading required package: wavelets

Loading required package: tictoc



In [2]:
workingdir <- getwd()
datadir <- file.path(workingdir, "Datos/daily_s")
eventdir <- file.path(workingdir, "Datos/daily_s/Event")
eventdir_old <- file.path(workingdir, "Datos/daily_s/Event/old")
lotdir <- file.path(workingdir, "Datos/daily_s/Lot")
lotdir_old <- file.path(workingdir, "Datos/daily_s/Lot/old")
qcdir <- file.path(workingdir, "Datos/daily_s/qc")
qcdir_old <- file.path(workingdir, "Datos/daily_s/qc/old")
resultsdir <- file.path(workingdir, "Resultados")

In [3]:
tensorflow::tf_gpu_configured()

TensorFlow built with CUDA:  TRUE 
GPU device name:  /device:GPU:0

[1] TRUE

Lectura de archivos de datos

In [6]:
# Leer archivo dataset desde Github:
url <- "https://raw.githubusercontent.com/garciasergio94y/RNN/LSTM_onehot_QC/Resultados/qc_data_sel_wl?token=GHSAT0AAAAAACDFXH5LJ7YOBL4BSAL5E4BKZDT5OMA"
qc_data_sel_wl <- read.csv(url)

# Guardar una copia local:
write.csv(qc_data_sel_wl, file = "qc_data_sel_wl", row.names = F, col.names = F) 


Warning message in write.csv(qc_data_sel_wl, file = "qc_data_sel_wl", row.names = F, :
“attempt to set 'col.names' ignored”


In [30]:
# Convertir fechas en formato POSIXct y niveles de QC en chr:

qc_data_sel_wl[,1] %<>% as.POSIXct(tz = "Europe/Madrid")
qc_data_sel_wl$NIVEL %<>% as.character()
head(qc_data_sel_wl)

,TIEMPO_QC,QC_RESULT,W1,W2,W3,W4,W5,W6,W7,V1,V2,V3,V4,V5,V6,V7,ANALIZADOR,CODIGO_PRUEBA,NIVEL
,<dttm>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>
1,2023-02-27 09:17:02,401.4,10.815059,-10.034408,-6.682440,4.332322,-2.0369932,-99999,-99999,373.8570,397.5725,372.7530,373.6333,96.13080,-99999,-99999,DXI800 num 1,CLC00815,3
2,2023-02-27 09:14:55,141.3,-3.409083,13.571425,-6.923823,7.590752,-0.8203487,-99999,-99999,384.8846,392.7571,374.0874,373.1011,94.00418,-99999,-99999,DXI800 num 1,CLC00815,2
3,2023-02-27 09:13:25,52.0,-10.542906,18.887730,-4.650879,10.071757,0.3812189,-99999,-99999,393.7813,386.3050,376.5056,372.6347,91.70756,-99999,-99999,DXI800 num 1,CLC00815,1
4,2023-02-24 09:18:42,421.7,14.753653,-7.126668,-1.256403,11.449636,1.5241625,-99999,-99999,384.6053,379.7896,379.7570,372.2544,89.24938,-99999,-99999,DXI800 num 1,CLC00815,3
5,2023-02-24 09:16:27,51.0,2.802629,-18.145619,0.978243,11.633733,2.5831021,-99999,-99999,378.3141,375.3619,383.5886,371.9828,86.63833,-99999,-99999,DXI800 num 1,CLC00815,1
6,2023-02-24 09:15:51,140.7,-18.803371,-1.326283,1.751535,10.662158,3.5343644,-99999,-99999,388.8738,374.5546,387.5806,371.8445,83.88591,-99999,-99999,DXI800 num 1,CLC00815,2


Definición de funciones para transformación de variables, normalización y secuenciación de series de datos:

In [8]:
# Función de normalización de las variables continuas y eliminación de coeficientes wavelet con valores NA. Se definen dos funciones, una para el set de training y otra para los sets de test y validación. La diferencia es que la de train devuelve vectores de medias y de desviaciones estándar que luego se usan como argumentos en las otras. De esta forma, el escalado y normalización se realiza con la misma media y desviación del set de entrenamiento para los otros dos datasets:

normalize_train <- function(data) {
   
  # Identificar las columnas que empiezan por W y la columna 'RESULTADO'
  wavelet_cols <- grep("^[WV]", colnames(data), value = TRUE)
  result_col <- "QC_RESULT"
  
  # Calcular medias y desviaciones estándar por grupos de pruebas y   analizador:
  means <- data %>%
    group_by(ANALIZADOR, CODIGO_PRUEBA) %>%
    summarise(across(c(result_col, wavelet_cols),
                     ~mean(., na.rm = T))) 
  
  std_devs <- data %>% 
    group_by(ANALIZADOR, CODIGO_PRUEBA) %>% 
    summarise(across(c(result_col, wavelet_cols),
                     ~sd(., na.rm = T)))
  
  # Normalizar las columnas de resultado y coeficientes wavelet
  norm_data <- data %>%
    group_by(ANALIZADOR, CODIGO_PRUEBA) %>%
    mutate(across(c(result_col, wavelet_cols), scale))%>%
    mutate(across(wavelet_cols,
                           ~ifelse(is.na(.), -99999, .))) %>%
    ungroup()
  
  # Devolver el dataframe escalado y las medias y desviaciones estándar:
  
  return(list(scaled_data = as.data.frame(norm_data), 
              means = means, stdev = std_devs))
}

In [9]:
# Función de normalización para los datos de validación y test, con eliminación de valores NA en los wavelets:

norm_test_val <- function(data, means, stds) {
  
  # Identificar las columnas numéricas y seleccionarlas
  wavelet_cols <- grep("^[WV]", colnames(data), value = TRUE)
  result_col <- "QC_RESULT"
  test_cols <- "CODIGO_PRUEBA"
  
  cols <- c(result_col, wavelet_cols, test_cols)
  
  # Seleccionar medias y desviaciones de resultados y coeficientes   wavelet:
  means_r <- train_n$means
  stds_r <- train_n$stdev
  means_wl <- train_n$means[wavelet_cols]
  stds_wl <- train_n$stdev[wavelet_cols]
  
  # Normalizar por cada prueba las columnas de resultado y coeficientes wavelet usando las medias y desviaciones estándar de train suministradas:
  means <- data %>%
    group_by(ANALIZADOR, CODIGO_PRUEBA) %>%
    left_join(means_r, by = c("ANALIZADOR", "CODIGO_PRUEBA"), 
              suffix = c(".val", ".means")) %>%
     ungroup()
   
  stds <- data %>%
    group_by(ANALIZADOR, CODIGO_PRUEBA) %>%
     left_join(stds_r, by = c("ANALIZADOR", "CODIGO_PRUEBA"),
               suffix = c(".val", ".sd")) %>%
    ungroup()
     
    
  norm_data <- data %>%
    mutate(., RESULTADO.st =
             (.[,result_col]-
                select(means,
                       starts_with(result_col) &
                       ends_with(".means"))) /
                select(stds,
                       starts_with(result_col) &
                       ends_with(".sd"))
           ) %>%
    mutate(WAVELETS.st =
             across(all_of(wavelet_cols),
                    ~ (. - select(means, 
                                  starts_with(cur_column()) &
                                    ends_with(".means"))[[1]]) /
                      select(stds,
                             starts_with(cur_column()) &
                               ends_with(".sd"))[[1]])
           ) %>%
    select(-c(result_col, wavelet_cols)) %>%
    unnest(c(WAVELETS.st, RESULTADO.st)) %>%
    relocate(starts_with("QC_"), .after = 1) %>%
    relocate(starts_with("W"), .after = 2) %>%
    relocate(starts_with("V"), .before = ANALIZADOR) 
    
  names(norm_data)[2]<-result_col
  
  return(as.data.frame(norm_data))
}



## Función ```code_cat_var```.

Función para convertir las variables categóricas tipo string en vectores numéricos y codificarlas (one hot), así como normalizar los valores de las variables continuas, creando una lista que retiene en un vector la información de los niveles de los factores

In [10]:
code_cat_var <- function(data) {
  # Eliminación de la columna CODIGO_PRUEBA si solo tiene una          categoría:
  if(length(unique(data$CODIGO_PRUEBA))==1){
    data %<>% select(., -CODIGO_PRUEBA)
    }
  
  
  # Detección de strings:
  cat_vars <- which(sapply(data, is.character))
  # Conversión a factores:
  data_cat <- lapply(data[,cat_vars], as.factor)
  # Extracción de niveles:
  levels <- lapply(data_cat, levels)
  
  # Codificar one-hot todas las variables factor:
  encoded_cols <- lapply(seq_along(data_cat), function(i) {
    cols <- model.matrix(~ factor(data_cat[[i]]) - 1)
    colnames(cols) <- paste0(names(data)[cat_vars[i]], "_",
                             levels[[i]])
    cols
    })
  
  # Unir columnas codificadas con conjunto de datos original:
  if (!"CODIGO_PRUEBA" %in% names(data)) {
    data_encoded <- bind_cols(data %>% 
                                select(-all_of(cat_vars)),
                                encoded_cols)
    } else {
      data_encoded <-  bind_cols(data %>%
                                   select(c(-all_of(cat_vars),
                                            -"CODIGO_PRUEBA")),
                                 encoded_cols)
      }
  
  return(list(data = data_encoded, levels = levels))
}

In [11]:
# Función para crear un índice de casos únicos de combinaciones de ANALIZADOR, CÓDIGO_PRUEBA y NIVEL:

unique_lab_cases <- function(df, categorical_vars) {
  unique_groups <- unique(df[categorical_vars])
  
  code <- apply(unique_groups, 1, function(row) {
    paste0(ifelse(row == 1, 1, 0), collapse = "")
  })
  unique_groups %<>% mutate(., code)
  coded_df <- df %>%
    merge(unique_groups, ., by = colnames(unique_groups)
          [-length(colnames(unique_groups))])
  coded_df <- coded_df[, c(colnames(df), "code")]
  
  return(coded_df)
}


In [12]:
# Función para creación de lotes de series temporales de longitud k (lookback)

create_lstm_data.2 <- 
  function(data, lookback, delay, min_index, max_index,
           shuffle = FALSE, batch_size, step = 1,
           predseries) {
  
  if (is.null(max_index)) max_index <- nrow(data) - delay - 1
  i <- min_index + lookback
  function() {
    if (shuffle) {
      rows <- sample(c((min_index+lookback):max_index), 
                     size = batch_size)
    } else {
      if (i + batch_size >= max_index)
        i <<- min_index + lookback
      rows <- c(i:min(i+batch_size, max_index))
      i <<- i + length(rows)
}
    samples <- array(0, dim = c(length(rows),
                                lookback / step,
                                dim(data)[[-1]]))
    targets <- array(0, dim = c(length(rows)))
    for (j in 1:length(rows)) {
      indices <- seq(rows[[j]] - lookback, rows[[j]],
                     length.out = dim(samples)[[2]])
      samples[j,,] <- data[indices, ]
      targets[[j]] <- data[rows[[j]] + delay,predseries]
    }
    list(samples, targets)
  }
}


In [13]:
index_train <- 1

train_gen <- function() {
  if (index_train > length(train_k)) {
    index_train <<- 1
  }
  
  sequence <- NULL
  counter <- 0
  
  while (is.null(sequence) && counter < length(train_k)) {
    sequence <- tryCatch(
      train_k[[index_train]](),
      error = function(e) {
        warning("Empty sequence error occurred in the generator.
                Trying the next one.")
        return(NULL)  # Devuelve NULL si ocurre un error en la función generadora
      }
    )
    
    index_train <<- index_train + 1
    counter <<- counter + 1
  }
  
  return(sequence)
}


index_val <- 1

val_gen <- function() {
  if (index_val > length(val_k)) {
    index_val <<- 1
  }
  
  sequence <- NULL
  counter <- 0
  
  while (is.null(sequence) && counter < length(val_k)) {
    sequence <- tryCatch(
      val_k[[index_val]](),
      error = function(e) {
        warning("Empty sequence error occurred in the generator.
                Trying the next one.")
        return(NULL)  # Devuelve NULL si ocurre un error en la función generadora
      }
    )
    
    index_val <<- index_val + 1
    counter <<- counter + 1
  }
  
  return(sequence)
}

In [31]:
# Se divide el dataset en 50% de datos para entrenamiento, 25% para test y 25% para validación.
# Para evitar fraccionar las unidades temporales diarias usamos percentiles de la secuencia temporal en días: 

time_point_50 <- as.POSIXct(as.Date(
  quantile(qc_data_sel_wl[[1]], 0.5)))

time_point_75 <- as.POSIXct(as.Date(
  quantile(qc_data_sel_wl[[1]], 0.75)))

time_point_50
time_point_75

             50% 
"2022-09-02 UTC" 

             75% 
"2022-12-02 UTC" 

In [32]:
# Muestra de entrenamiento:
train_n <- qc_data_sel_wl %>%
  subset(.[[1]] <= time_point_75) %>%
  # Normalización del dataset y eliminación de coeficientes
  # wavelet con valor NA:
  normalize_train()

`summarise()` has grouped output by 'ANALIZADOR'. You can override using the
`.groups` argument.
`summarise()` has grouped output by 'ANALIZADOR'. You can override using the
`.groups` argument.


In [33]:
# Muestra de validación:
val_n <- qc_data_sel_wl %>%
  subset(
    #.[[1]] > time_point_50 &
    .[[1]] > time_point_75) %>%
  # Normalización del dataset y eliminación de coeficientes
  # wavelet con valor NA:
  norm_test_val(means = train$means, stds = train$stdev)

In [34]:
# Codificación onehot:

train_code <- code_cat_var(train_n$scaled_data)

val_code <- code_cat_var(val_n)

In [35]:
# Creación de series temporales de tamaño k con los datos de training:

# Creación de los códigos identificativos únicos de combinaciones de variables categóricas para el set de training usando la función unique_lab_cases e indicando las posiciones de las variables categóricas en train_code$data que se deseen incluir (en este caso el analizador): 

train_indx <- unique_lab_cases(train_code$data,
                               categorical_vars = c(17:48))[,-1]

# Extraemos los códigos para realizar un diccionario de traducción de códigos únicos a índices numéricos:
code_values <- unique(train_indx[,"code"])
code_dictionary <- data.frame(code = code_values)
code_dictionary %<>%  
  mutate(., index = as.numeric(factor(code_values, 
                                      levels = code_values)))

# Sustituimos los códigos de train_indx por sus índices numéricos:
train_indx$index <- code_dictionary$index[match(train_indx$code, code_dictionary$code)]
index_values <- unique(train_indx[,"index"])
train_indx$code <- NULL

# Hiperparámetros para generar las series temporales:
lookback <- 30 # Longitud de la secuencia previa al target.
delay <- 1 # Escoge como target el valor k+1 en el futuro.
batch_size <- 100 # Tamaño de batch.
min_index <- 1 # Indice de partida de la serie temporal.
predseries <- 1 # Posición en el dataset de la variable a predecir (QC_RESULT)

# Creación de las series temporales de longitud k por categorías de combinaciones únicas de analizador, código de prueba y nivel de QC:
 
train_k <- lapply(index_values, function(idx) {
  create_lstm_data.2(
    data = as.matrix(train_indx[train_indx[,"index"] == idx,
                                - length(train_indx)]),
    lookback = lookback,
    delay = delay,
    batch_size = batch_size,
    min_index = min_index,
    max_index = 
      nrow(as.matrix(
        train_indx[train_indx[,"index"] == idx,]))-delay-1,
    predseries = predseries)
})

# Cálculo del número de steps de training:
train_steps <- round((nrow(train_code$data)-lookback) / batch_size)

In [36]:
val_indx <- unique_lab_cases(val_code$data, 
                             categorical_vars = c(17:48))[,-1]

# Usamos el diccionario de códigos creado anteriormente: 

val_indx$index <- code_dictionary$index[match(val_indx$code, code_dictionary$code)]
index_values <- unique(val_indx[,"index"])
val_indx$code <- NULL

val_k <- lapply(index_values, function(idx) {
  create_lstm_data.2(
    data = as.matrix(val_indx[val_indx[,"index"] == idx,
                              - length(val_indx)]),
    lookback = lookback,
    delay = delay,
    batch_size = batch_size,
    min_index = min_index,
    max_index = 
      nrow(as.matrix(val_indx[val_indx[,"index"] == idx,]))-delay-1,
    predseries = predseries)
})

# Cálculo del número de steps de validación:
val_steps <- round((nrow(val_code$data)-lookback) / batch_size)

In [37]:
# Funciones generadoras de series para train y validation, Estas funciones generan 
# de manera indefinida series con los datos de entrenamiento y validación, usando 
# de manera consecutiva las funciones generadoras almacenadas junto a los subconjuntos 
# de datos con las 160 posibles combinaciones de ANALIZADOR, CÓDIGO_PRUEBA y NIVEL de QC
# sobre los que cada una tiene que actuar. Si se llega a la generación número 160 
# se repite el proceso desde la primera.  

index_train <- 1

train_gen <- function() {
  if (index_train > length(train_k)) {
      index_train <<- 1
  }
    
  sequence <- train_k[[index_train]]()
  index_train <<- index_train + 1
  
  return(sequence)
}

index_val <- 1

val_gen <- function() {
  if (index_val > length(val_k)) {
    index_val <<- 1
  }

  sequence <- val_k[[index_val]]()
  index_val <<- index_val + 1
  
  return(sequence)
}

In [ ]:
# Función para calcular el MAE promedio para un modelo basado en predecir los targets 
# del set de validación con las mismas muestras de validación, mediante la iteración 
# de lotes de muestras. Esto sería equivalente a un modelo que predice el resultado 
# futuro del QC con el actual y constituye el punto de partida para mejorar (common sense baseline).

evaluate_naive_method <- function() {
  batch_maes <- c()
  for (step in 1:val_steps) {
    c(samples, targets) %<-% val_gen()
    preds <- samples[,dim(samples)[[2]],2]
    mae <- mean(abs(preds - targets))
    batch_maes <- c(batch_maes, mae)
  }
  print(mean(batch_maes))
}

maes <- evaluate_naive_method()
cbind(train_n$stdev[[3]], maes*train_n$stdev[[3]])


[1] 3.975875


CLC00033,53.5918894310736
CLC00034,5.57748042439256
CLC00061,183.733326859503
CLC00415,381.81579837991
CLC00544,88.998850198785
CLC00638,28.4827195332944
CLC00805,681.912102074728
CLC00807,222.808695650046
CLC00811,17.7739267983337
CLC00813,319.102338561277
CLC00814,66.3543590252383


In [39]:
set.seed(123)

model_2 <- keras_model_sequential() %>%
    layer_lstm(units = 32, dropout=0.1, recurrent_dropout=0.5,
              #return_sequences = TRUE,
              input_shape = list(NULL, dim(train_code$data[,-1])[[-1]])) %>%
    #layer_lstm(units = 64, activation="relu", 
               #dropout=0.1, recurrent_dropout=0.5) %>%
    layer_dense(units = 1)

In [40]:
rmse <- function(y_pred, y_true) {
  backend <- keras::backend()
  sq_diff <- backend$square(y_pred - y_true)
  mean_sq_diff <- backend$mean(sq_diff, axis = -1)
  rmse_value <- backend$sqrt(mean_sq_diff)
  rmse_value <- backend$cast(rmse_value, dtype = "float32")
  rmse_value
}


In [41]:
model_2 %>% compile(
    optimizer = "rmsprop",
    loss="mse", 
    metrics = list("mse", "mae")
)

callbacks <- list(
  callback_model_checkpoint(file.path(resultsdir, "lstm_2.keras",
                                     save_best_only = TRUE)))

In [42]:
# Entrenar el modelo:
history_2 <- model_2 %>% fit_generator(
  generator = train_gen,
  steps_per_epoch = train_steps,
  epochs = 20,
  validation_data = val_gen,
  validation_steps = val_steps,
  callbacks = callbacks
)

Warning message in fit_generator(., generator = train_gen, steps_per_epoch = train_steps, :
“`fit_generator` is deprecated. Use `fit` instead, it now accept generators.”


In [1]:
plot(history_2)

ERROR: ignored